# LSTM

In practice, it is quite difficult to train RNNs for tasks that require a network to make use of information distant from the current point of processing. Despite having access to the entire preceding sequence, the information encoded in hidden states tends to be fairly local, more relevant to the most recent parts of the input sequence and recent decisions. Yet distant information is critical to many language applications. 

Consider the following example in the context of language modeling.

<pre>
The flights the airline was canceling were full.
</pre>

Assigning a high probability to _was_ following _airline_ is straightforward since _airline_ provides a strong local context for the singular agreement. However, assigning an appropriate probability to _were_ is quite difficult, not only because the plural _flights_ is quite distant, but also because the singular noun _airline_ is closer in the intervening context. Ideally, a network should be able to retain the distant information about plural _flights_ until it is needed, while still processing the intermediate parts of the sequence correctly.

One reason for the inability of RNNs to carry forward critical information is that the hidden layers, and, by extension, the weights that determine the values in the hidden layer, are being asked to perform two tasks simultaneously: provide information useful for the current decision, and updating and carrying forward information required for future decisions.

A second difficulty with training RNNs arises from the need to backpropagate the error signal back through time. Recall that the hidden layer at time $t$ contributes to the loss at the next time step since it takes part in that calculation. As a result, during the backward pass of training, the hidden layers are subject to repeated multiplications, as determined by the length of the sequence. A frequent result of this process is that the gradients are eventually driven to zero, a situation RNNS AND LSTM called the **vanishing gradients** problem.

To address these issues, more complex network architectures have been designed to explicitly manage the task of maintaining relevant context over time, by enabling the network to learn to forget information that is no longer needed and to remember information required for decisions still to come.

The most commonly used such extension to RNNs is the **long short-term memory** (LSTM) network (Hochreiter and Schmidhuber, 1997). LSTMs divide the context management problem into two subproblems: removing information no longer needed from the context, and adding information likely to be needed for later decision making. The key to solving both problems is to learn how to manage this context rather than hard-coding a strategy into the architecture. LSTMs accomplish this by first adding an explicit context layer to the architecture (in addition to the usual recurrent hidden layer), and through the use of specialized neural units that make use of gates to control the flow of information into and out of the units that comprise the network layers. These gates are implemented through the use of additional weights that operate sequentially on the input, and previous hidden layer, and previous context layers.

The gates in an LSTM share a common design pattern; each consists of a feedforward layer, followed by a sigmoid activation function, followed by a pointwise multiplication with the layer being gated. The choice of the sigmoid as the activation function arises from its tendency to push its outputs to either 0 or 1. Combining this with a pointwise multiplication has an effect similar to that of a binary mask. Values in the layer being gated that align with values near 1 in the mask are passed through nearly unchanged; values corresponding to lower values are essentially erased.

The first gate we’ll consider is the **forget gate**. The purpose of this gate is to delete information from the context that is no longer needed. The forget gate computes a weighted sum of the previous state’s hidden layer and the current input and passes that through a sigmoid. This mask is then multiplied element-wise by the context vector to remove the information from context that is no longer required. Element-wise multiplication of two vectors (represented by the operator $\odot$, and sometimes called the **Hadamard product**) is the vector of the same dimension as the two input vectors, where each element $i$ is the product of element $i$ in the two input vectors:

$$
f_t = \sigma(U_fh_{t-1} + W_fx_t) \\
k_t = c_{t-1} \odot f_t
$$

The next task is to compute the actual information we need to extract from the previous hidden state and current inputs — the same basic computation we’ve been using for all our recurrent networks.

$$
g_t = \text{tanh}(U_gh_{t-1} + W_gx_t)
$$

Next, we generate the mask for the **add gate** to select the information to add to the current context.

$$
i_t = \sigma(U_ih_{t-1} + W_ix_t) \\
j_t = g_t \odot i_t
$$

Next, we add this to the modified context vector to get our new context vector.

$$
c_t = j_t + k_t
$$

The final gate we’ll use is the **output gate** which is used to decide what information is required for the current hidden state (as opposed to what information needs to be preserved for future decisions).

$$
o_t = \sigma(U_oh_{t-1} + W_ox_t) \\
h_t = o_t \odot \text{tanh}(c_t)
$$

Figure below illustrates the complete computation for a single LSTM unit. Given the appropriate weights for the various gates, an LSTM accepts as input the context layer, and hidden layer from the previous time step, along with the current input vector. It then generates updated context and hidden vectors as output.

It is the hidden state, $h_t$, that provides the output for the LSTM at each time step. This output can be used as the input to subsequent layers in a stacked RNN, or at the final layer of a network $h_t$ can be used to provide the final output of the LSTM.


<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/lstm_architecture.png"  style="width:800px;object-fit:cover;"/>
</div>